In [1]:
import pandas as pd
import sqlite3

In [2]:
!pip install openpyxl

In [3]:
from google.colab import files
uploaded = files.upload()

Saving Flor_martData.xlsx to Flor_martData.xlsx


In [4]:
excel_path = list(uploaded.keys())[0]
df = pd.read_excel(excel_path, engine='openpyxl')

In [5]:
print("Columns in your dataset:\n", df.columns.tolist())

Columns in your dataset:
 ['City', 'Customer Age', 'Customer Name', 'Customer Segment', 'Discount', 'Number of Records', 'Order Date', 'Order ID', 'Order Priority', 'Order Quantity', 'Product Base Margin', 'Product Category', 'Product Container', 'Product Name', 'Product Sub-Category', 'Profit', 'Region', 'Row ID', 'Sales', 'Ship Date', 'Ship Mode', 'Shipping Cost', 'State', 'Unit Price', 'Zip Code', 'Profit percent', 'Lead Time', 'Delivery Bucket', 'unit sold']


In [6]:
conn = sqlite3.connect(':memory:')
df.to_sql('flormart', conn, index=False)

cursor = conn.cursor()

columns_to_check = ["Profit", "Sales", "Ship Date", "State", "Zip Code", "Profit percent"]
for col in columns_to_check:
    query = f"SELECT COUNT(*) FROM flormart WHERE \"{col}\" IS NULL"
    cursor.execute(query)
    result = cursor.fetchone()[0]
    print(f"Number of nulls in '{col}': {result}")

conn.close()

Number of nulls in 'Profit': 0
Number of nulls in 'Sales': 0
Number of nulls in 'Ship Date': 0
Number of nulls in 'State': 0
Number of nulls in 'Zip Code': 0
Number of nulls in 'Profit percent': 0


In [9]:
conn = sqlite3.connect(':memory:')
df.to_sql('flormart', conn, index=False)

cursor = conn.cursor()

# Example date range query
query = """
SELECT MIN("Order Date") AS min_date, MAX("Order Date") AS max_date
FROM flormart
"""

cursor.execute(query)
results = cursor.fetchone()

# Print the results (or process them further)
print(f"Minimum Order Date: {results[0]}, Maximum Order Date: {results[1]}")


conn.close()

Minimum Order Date: 2012-01-01 00:00:00, Maximum Order Date: 2015-12-30 00:00:00


In [11]:
conn = sqlite3.connect(':memory:')
df.to_sql('flormart', conn, index=False)

cursor = conn.cursor()

query = """
 SELECT [Order ID], COUNT(*) AS count
    FROM flormart
    GROUP BY [Order ID]
    HAVING COUNT(*) > 1
"""
cursor.execute(query)
results = cursor.fetchone()

print("\nDuplicate Order IDs:\n", results)
conn.close()


Duplicate Order IDs:
 (32, 4)


In [14]:
conn = sqlite3.connect(':memory:')
df.to_sql('flormart', conn, index=False)

cursor = conn.cursor()

# Calculate mean and standard deviation of Profit
query_stats = """
SELECT AVG(Profit)
FROM flormart
"""
cursor.execute(query_stats)
mean_profit = cursor.fetchone()[0]

query_stdev = f"""
SELECT SQRT(AVG((Profit - {mean_profit}) * (Profit - {mean_profit})))
FROM flormart
"""
cursor.execute(query_stdev)
stdev_profit = cursor.fetchone()[0]

# Define a threshold for outliers (e.g., 3 standard deviations)
threshold = 3

# Identify outliers using SQL
query_outliers = f"""
SELECT COUNT(*)
FROM flormart
WHERE Profit < ({mean_profit} - {threshold} * {stdev_profit})
OR Profit > ({mean_profit} + {threshold} * {stdev_profit})
"""

cursor.execute(query_outliers)
num_outliers = cursor.fetchone()[0]

# Get total number of rows
query_total_rows = """
SELECT COUNT(*)
FROM flormart
"""
cursor.execute(query_total_rows)
total_rows = cursor.fetchone()[0]

# Calculate percentage of outliers
percentage_outliers = (num_outliers / total_rows) * 100 if total_rows > 0 else 0

print(f"Number of potential Profit Outliers (more than {threshold} standard deviations from the mean): {num_outliers}")
print(f"Percentage of Profit Outliers: {percentage_outliers:.2f}%")

conn.close()

Number of potential Profit Outliers (more than 3 standard deviations from the mean): 178
Percentage of Profit Outliers: 2.12%
